In [2]:
"""
If necessary, uncomment and install:
"""
#!pip install wbdata
#!pip install cufflinks
#!pip install iso3166

import iso3166 #iso3166.countries.get('country details')
import wbdata
import cufflinks as cf
import pandas as pd
import numpy as np
import plotly
import matplotlib.pyplot as plt
import seaborn as sns
cf.go_offline()

  Using cached wbdata-0.3.0-py3-none-any.whl (14 kB)
  Using cached typing_extensions-4.0.1-py3-none-any.whl (22 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow 2.6.2 requires h5py~=3.1.0, but you'll have h5py 3.3.0 which is incompatible.
tensorflow 2.6.2 requires numpy~=1.19.2, but you'll have numpy 1.21.5 which is incompatible.
tensorflow 2.6.2 requires six~=1.15.0, but you'll have six 1.16.0 which is incompatible.
tensorflow 2.6.2 requires typing-extensions~=3.7.4, but you'll have typing-extensions 4.0.1 which is incompatible.


/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning:

The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.



In [10]:
import wbdata

# Return list of all country/region codes:
wbdata.get_country()

# Return list matching a query term:
wbdata.search_countries("Syrian Arab Republic")

## Try your own search!
# wbdata.search_countries("")

id    name
----  --------------------
SYR   Syrian Arab Republic

In [18]:
wbdata.get_source()

  id  name
----  --------------------------------------------------------------------
   1  Doing Business
   2  World Development Indicators
   3  Worldwide Governance Indicators
   5  Subnational Malnutrition Database
   6  International Debt Statistics
  11  Africa Development Indicators
  12  Education Statistics
  13  Enterprise Surveys
  14  Gender Statistics
  15  Global Economic Monitor
  16  Health Nutrition and Population Statistics
  18  IDA Results Measurement System
  19  Millennium Development Goals
  20  Quarterly Public Sector Debt
  22  Quarterly External Debt Statistics SDDS
  23  Quarterly External Debt Statistics GDDS
  24  Poverty and Equity
  25  Jobs
  27  Global Economic Prospects
  28  Global Financial Inclusion
  29  The Atlas of Social Protection: Indicators of Resilience and Equity
  30  Exporter Dynamics Database – Indicators at Country-Year Level
  31  Country Policy and Institutional Assessment
  32  Global Financial Development
  33  G20 Financial Inclus

In [40]:
SOURCE = 40 # "Population estimates and projections

indicators = wbdata.get_indicator(source=SOURCE)
indicators

id                 name
-----------------  -------------------------------------------------------------------
SH.DTH.0509        Number of deaths ages 5-9 years
SH.DTH.1014        Number of deaths ages 10-14 years
SH.DTH.1519        Number of deaths ages 15-19 years
SH.DTH.2024        Number of deaths ages 20-24 years
SH.DTH.IMRT        Number of infant deaths
SH.DTH.IMRT.FE     Number of infant deaths, female
SH.DTH.IMRT.MA     Number of infant deaths, male
SH.DTH.MORT        Number of under-five deaths
SH.DTH.MORT.FE     Number of under-five deaths, female
SH.DTH.MORT.MA     Number of under-five deaths, male
SH.DTH.NMRT        Number of neonatal deaths
SH.DYN.0509        Probability of dying among children ages 5-9 years (per 1,000)
SH.DYN.1014        Probability of dying among adolescents ages 10-14 years (per 1,000)
SH.DYN.1519        Probability of dying among adolescents ages 15-19 years (per 1,000)
SH.DYN.2024        Probability of dying among youth ages 20-24 years (per 1,000)

In [35]:
import iso3166 #iso3166.countries.get('country details')
year = 2015 #1960 to 2050 (projections)
group = 'all' #males/females/all
age_lower = 10 #rounds to nearest 5 (actual range: 10 to 65)
age_upper = 80
location = 'Syrian Arab Republic'
def get_dataframe(year = '2018', group='all', age_lower=0, age_upper=100, location ='world'):
    country_code = "WLD"
    if location != 'world':
        country_code = iso3166.countries.get(location).alpha3
    age_ranges = []
    for i in range(age_lower, age_upper, 5):
        age_ranges.append(f"{i:02d}"+f"{i+4:02d}")
    age_ranges.append("80UP")
    
    female_variables = {"SP.POP."+age_range+".FE":"Females "+age_range for age_range in age_ranges}
    female_df = wbdata.get_dataframe(female_variables,country=country_code)
    female_data = female_df.query("date=='{}'".format(year)).sum(axis=0).tolist()
    
    male_variables = {"SP.POP."+age_range+".MA":"Males "+age_range for age_range in age_ranges}
    male_df = wbdata.get_dataframe(male_variables,country=country_code)
    male_data = male_df.query("date=='{}'".format(year)).sum(axis=0).tolist()
    
    age_input = [i[:2]+'-'+i[2:] for i in age_ranges]
    
    df = pd.DataFrame({
        'Country': location,
        'Year': year,
        'Age': age_input,
        'Female': female_data,
        'Male': male_data
    })
    return df

get_dataframe(year, group, age_lower, age_upper, location)

In [37]:
pop_df = get_dataframe(year, group, age_lower, age_upper, location)
pop_df

,Country,Year,Age,Female,Male
0,Syrian Arab Republic,2015,10-14,891220.0,935115.0
1,Syrian Arab Republic,2015,15-19,886157.0,937570.0
2,Syrian Arab Republic,2015,20-24,822850.0,874225.0
3,Syrian Arab Republic,2015,25-29,838386.0,881533.0
4,Syrian Arab Republic,2015,30-34,777094.0,777856.0
5,Syrian Arab Republic,2015,35-39,621934.0,600516.0
6,Syrian Arab Republic,2015,40-44,476829.0,458170.0
7,Syrian Arab Republic,2015,45-49,394387.0,380835.0
8,Syrian Arab Republic,2015,50-54,340216.0,326070.0
9,Syrian Arab Republic,2015,55-59,267537.0,255496.0


In [39]:
pop_df['Age']

0     10-14
1     15-19
2     20-24
3     25-29
4     30-34
5     35-39
6     40-44
7     45-49
8     50-54
9     55-59
10    60-64
11    65-69
12    70-74
13    75-79
14    80-UP
Name: Age, dtype: object

In [41]:
test = pd.Series([891220, np.nan, np.nan, np.nan, np.nan, 886157])
test.interpolate()

0    891220.0
1    890207.4
2    889194.8
3    888182.2
4    887169.6
5    886157.0
dtype: float64

In [23]:
test = pd.Series([10, 20, np.nan, 40, 50])
test.interpolate()

0    10.0
1    20.0
2    30.0
3    40.0
4    50.0
dtype: float64

In [24]:
test2 = pd.Series([50, 80, np.nan, 40, 60])
test2.interpolate()

0    50.0
1    80.0
2    60.0
3    40.0
4    60.0
dtype: float64

In [25]:
test3 = pd.Series([10, np.nan, np.nan, np.nan, np.nan, 80])
test3.interpolate()

0    10.0
1    24.0
2    38.0
3    52.0
4    66.0
5    80.0
dtype: float64